In [5]:
import glob
import ntpath
import pandas
import helpers
import settings
import numpy

CUBE_IMGTYPE_SRC = "_i"
CUBE_SIZE=32

In [10]:
src_dir =  "D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels/"
dst_dir =  "D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels_avg/"

#逐一遍历所有病人的positive label文件
for patient_index, csv_file in enumerate(glob.glob(src_dir + "*_annos_pos_lidc.csv")):
    print(patient_index)
    print(csv_file)
    ############################################################################
    #读取数据
    patient_id = ntpath.basename(csv_file).replace("_annos_pos_lidc.csv", "") 
    df_annos = pandas.read_csv(csv_file)
    if len(df_annos) == 0:
        continue    
    df_annos = df_annos[['coord_x','coord_y','coord_z','diameter','texture']]
    record_Number = len(df_annos)
    ############################################################################
    #统计肺结节数量
    images = helpers.load_patient_images(patient_id, settings.LUNA16_EXTRACTED_IMAGE_DIR, "*" + CUBE_IMGTYPE_SRC + ".png")
    coord=numpy.array( df_annos.loc[:,['coord_x','coord_y','coord_z'] ] )
    confidence=numpy.array( df_annos.loc[:,['diameter']] )[:,0]
    AB=toAB(coord,images.shape)
    keep=NMS(AB,confidence,0.5)
    df_annos_temp = df_annos.loc[keep].reset_index(drop=True)
    nodule_Number = len(df_annos_temp)
    ############################################################################
    #获取肺结节的平均数据
    temp_df = [] #存放平均之后的nodule信息
    for i in keep:
        temp_id = [] #存放重复检测到nodule的编号
        for j in range(record_Number):
            IoU = get_IOU(AB[i],AB[j])
            if IoU > 0.5:
                temp_id.append(j)
        coord_x = 0; coord_y = 0; coord_z = 0; diameter = 0; texture = 0
        for k in temp_id:
            coord_x += df_annos.iloc[k]["coord_x"]
            coord_y += df_annos.iloc[k]["coord_y"]
            coord_z += df_annos.iloc[k]["coord_z"]
            diameter += df_annos.iloc[k]["diameter"]
            texture += df_annos.iloc[k]["texture"]
        coord_x = coord_x / len(temp_id); coord_y = coord_y / len(temp_id); coord_z = coord_z / len(temp_id); diameter = diameter / len(temp_id); texture = texture / len(temp_id) 
        temp = [round(coord_x,4),round(coord_y,4),round(coord_z,4),round(diameter,4),round(texture,4)]
        temp_df.append(temp)
    columns = ["coord_x","coord_y","coord_z","diameter","texture"]
    temp_df = pandas.DataFrame(temp_df, columns=columns)
    ############################################################################
    #保存成csv文件
    temp_df.to_csv(dst_dir  +  ntpath.basename(csv_file), index=False)

0
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860_annos_pos_lidc.csv
1
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100332161840553388986847034053_annos_pos_lidc.csv
2
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793540579077826395208_annos_pos_lidc.csv
3
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100530488926682752765845212286_annos_pos_lidc.csv
4
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100620385482151095585000946543_annos_pos_lidc.csv
5
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405_annos_pos_lidc.csv
6
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.627

54
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.120196332569034738680965284519_annos_pos_lidc.csv
55
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.120842785645314664964010792308_annos_pos_lidc.csv
56
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.121108220866971173712229588402_annos_pos_lidc.csv
57
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.121391737347333465796214915391_annos_pos_lidc.csv
58
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.121805476976020513950614465787_annos_pos_lidc.csv
59
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.121824995088859376862458155637_annos_pos_lidc.csv
60
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.

109
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.139444426690868429919252698606_annos_pos_lidc.csv
110
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.139577698050713461261415990027_annos_pos_lidc.csv
111
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.139595277234735528205899724196_annos_pos_lidc.csv
112
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.139713436241461669335487719526_annos_pos_lidc.csv
113
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.139889514693390832525232698200_annos_pos_lidc.csv
114
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.140239815496047437552471323962_annos_pos_lidc.csv
115
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

168
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.160124400349792614505500125883_annos_pos_lidc.csv
169
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.160216916075817913953530562493_annos_pos_lidc.csv
170
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.160586340600816116143631200450_annos_pos_lidc.csv
171
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.161002239822118346732951898613_annos_pos_lidc.csv
172
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.161067514225109999586362698069_annos_pos_lidc.csv
173
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.161073793312426102774780216551_annos_pos_lidc.csv
174
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

222
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.176869045992276345870480098568_annos_pos_lidc.csv
223
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.177086402277715068525592995222_annos_pos_lidc.csv
224
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.177252583002664900748714851615_annos_pos_lidc.csv
225
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.177685820605315926524514718990_annos_pos_lidc.csv
226
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.177785764461425908755977367558_annos_pos_lidc.csv
227
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.177888806135892723698313903329_annos_pos_lidc.csv
228
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

...

660
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.330643702676971528301859647742_annos_pos_lidc.csv
661
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.331211682377519763144559212009_annos_pos_lidc.csv
662
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.332453873575389860371315979768_annos_pos_lidc.csv
663
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.332829333783605240302521201463_annos_pos_lidc.csv
664
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.333145094436144085379032922488_annos_pos_lidc.csv
665
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.333319057944372470283038483725_annos_pos_lidc.csv
666
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

714
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.428038562098395445838061018440_annos_pos_lidc.csv
715
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.430109407146633213496148200410_annos_pos_lidc.csv
716
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.436403998650924660479049012235_annos_pos_lidc.csv
717
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.438308540025607517017949816111_annos_pos_lidc.csv
718
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.439153572396640163898529626096_annos_pos_lidc.csv
719
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.440226700369921575481834344455_annos_pos_lidc.csv
720
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

768
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.618434772073433276874225174904_annos_pos_lidc.csv
769
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417051974713149104919_annos_pos_lidc.csv
770
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.621916089407825046337959219998_annos_pos_lidc.csv
771
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.622243923620914676263059698181_annos_pos_lidc.csv
772
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.624425075947752229712087113746_annos_pos_lidc.csv
773
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.625270601160880745954773142570_annos_pos_lidc.csv
774
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

822
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.780558315515979171413904604168_annos_pos_lidc.csv
823
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.792381786708289670758399079830_annos_pos_lidc.csv
824
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.797637294244261543517154417124_annos_pos_lidc.csv
825
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.799582546798528864710752164515_annos_pos_lidc.csv
826
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.801945620899034889998809817499_annos_pos_lidc.csv
827
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.802595762867498341201607992711_annos_pos_lidc.csv
828
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

877
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.965620538050807352935663552285_annos_pos_lidc.csv
878
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.969607480572818589276327766720_annos_pos_lidc.csv
879
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.970264865033574190975654369557_annos_pos_lidc.csv
880
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.970428941353693253759289796610_annos_pos_lidc.csv
881
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.975254950136384517744116790879_annos_pos_lidc.csv
882
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.14519.5.2.1.6279.6001.975426625618184773401026809852_annos_pos_lidc.csv
883
D:/jupyter-notebook/LungCancerPredict/LIDC_Annotations_3/_labels\1.3.6.1.4.1.1

In [6]:
def get_IOU(data1,data2):      
    
    data1_A = data1[0,:]
    data1_B = data1[1,:]
    data2_A = data2[0,:]
    data2_B = data2[1,:]
    
    areas = (data1_B[0] - data1_A[0] + 1) * (data1_B[1] - data1_A[1] + 1) * (data1_B[2] - data1_A[2] + 1)   

    #计算窗口i与其他所有窗口的交叠部分的面积  
    A_X = numpy.maximum(data1_A[0], data2_A[0])  
    A_Y = numpy.maximum(data1_A[1], data2_A[1])  
    A_Z = numpy.maximum(data1_A[2], data2_A[2])  

    B_X = numpy.minimum(data1_B[0], data2_B[0])  
    B_Y = numpy.minimum(data1_B[1], data2_B[1])  
    B_Z = numpy.minimum(data1_B[2], data2_B[2])  
    
    x = numpy.maximum(0.0, B_X - A_X +1 )   
    y = numpy.maximum(0.0, B_Y - A_Y +1 )     
    z = numpy.maximum(0.0, B_Z - A_Z +1 )  
    
    inter = x * y * z  #交/并得到iou值 
    
    IoU = inter / (areas + areas - inter)
    
    return IoU

In [7]:
def NMS(data,confidence,threshold):      
    
    A=data[:,0,:]
    B=data[:,1,:]
   
    scores = confidence  #bbox打分      

    areas = (B[:,0] - A[:,0] + 1) * (B[:,1] - A[:,1] + 1) * (B[:,2] - A[:,2] + 1)   
    
    #打分从大到小排列，取index      
    order = scores.argsort()[::-1]  

    #keep为最后保留的边框      
    keep = []     
    while order.size > 0: 
        
        #order[0]是当前分数最大的窗口，肯定保留          
        i = order[0]         
        keep.append(i)  
        
        #计算窗口i与其他所有窗口的交叠部分的面积  
        A_X = numpy.maximum(A[i,0], A[order[1:],0])  
        A_Y = numpy.maximum(A[i,1], A[order[1:],1])  
        A_Z = numpy.maximum(A[i,2], A[order[1:],2])  
        
        B_X = numpy.minimum(B[i,0], B[order[1:],0])  
        B_Y = numpy.minimum(B[i,1], B[order[1:],1])  
        B_Z = numpy.minimum(B[i,2], B[order[1:],2])  
          
        x = numpy.maximum(0.0, B_X - A_X +1 )   
        
        y = numpy.maximum(0.0, B_Y - A_Y +1 )     

        z = numpy.maximum(0.0, B_Z - A_Z +1 )  
              
        inter = x * y * z  #交/并得到iou值 
          
        IoU = inter / (areas[i] + areas[order[1:]] - inter) 

        #index为所有与窗口i的iou值小于threshold值的窗口的index，其他窗口此次都被窗口i吸收      
        index = numpy.where(IoU <= threshold)[0]  
        
        #order里面只保留与窗口i交叠面积小于threshold的那些窗口，由于IoU长度比order长度少1(不包含i)，所以index+1对应到保留的窗口     
        order = order[index + 1]          
    
    return keep

In [8]:
#由中心点的坐标，求bbox的A和B 
def toAB(coord,shape):
    
    AB=[]
    
    for i in range(coord.shape[0]):
        
        temp=[]
        
        centr_X=int(coord[i][0] * shape[1])
        centr_Y=int(coord[i][1] * shape[2])
        centr_Z=int(coord[i][2] * shape[0])
        
        A=[ centr_X-CUBE_SIZE/2 , centr_Y-CUBE_SIZE/2 , centr_Z-CUBE_SIZE/2 ]  #A永远是坐标值较小的那边
        B=[ centr_X+CUBE_SIZE/2 , centr_Y+CUBE_SIZE/2 , centr_Z+CUBE_SIZE/2 ]
        
        temp=[]
        temp.append(A)
        temp.append(B)
    
        AB.append(temp)
        
    return numpy.array(AB)